In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [3]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = np.ndarray(shape=(1, 11))
# property_stats[0][CRIM_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()



In [4]:
features.mean()

CRIM         3.613524
ZN          11.363636
CHAS         0.069170
NOX          0.554695
RM           6.284634
DIS          3.795043
RAD          9.549407
TAX        408.237154
PTRATIO     18.455534
B          356.674032
LSTAT       12.653063
dtype: float64

In [5]:
print(type(features.mean().values))
features.mean().values.shape

<class 'numpy.ndarray'>


(11,)

In [6]:
property_stats = features.mean().values.reshape(1, 11)

In [7]:
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [8]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [9]:
def get_log_estimate(nr_rooms,
                    students_per_classroom,
                    next_to_river=False,
                    high_confidence=True):
    
    #Configure Property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    
    #Make Prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    #Calc Range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval= 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [10]:
estimated_log_price, log_upper_bound, log_lower_bound, interval = get_log_estimate(9, 15, next_to_river=False,
                                                                                   high_confidence=False)

In [11]:
np.median(boston_dataset.target)

21.2

## Only for rise in inflation 


In [12]:
# House inflation rate rate between year 1970 and 2020 is 567%
inflation = 5.67
estimated_price = round((np.e**estimated_log_price)*1000, 2)                           #In 1970
estimated_price_latest =  round(estimated_price + inflation*estimated_price, 2)        #In 2020

upper_bound = round((np.e**log_upper_bound)*1000, 2)                                   #In 1970
upper_bound_latest =  round(upper_bound + inflation*upper_bound, 2)                    #In 2020

lower_bound = round((np.e**log_lower_bound)*1000, 2)                                   #In 1970
lower_bound_latest = round(lower_bound + inflation*lower_bound, 2)                     #In 2020

print('Price in 2020 for house esimate: $',estimated_price_latest)
print('Price in 2020 for maximum house price: $',upper_bound_latest)
print('Price in 2020 for minimum house price: $',lower_bound_latest)


print('Price in 1970 for house estimate: $',estimated_price)
print('Price in 1970 for maximum house price: $',upper_bound)
print('Price in 1970 for minimum house price: $',lower_bound)

Price in 2020 for house esimate: $ 200416.16
Price in 2020 for maximum house price: $ 241750.95
Price in 2020 for minimum house price: $ 166148.77
Price in 1970 for house estimate: $ 30047.4
Price in 1970 for maximum house price: $ 36244.52
Price in 1970 for minimum house price: $ 24909.86


## For rise in Inflation and other Property prices 

In [13]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target)

estimated_price = round((np.e**estimated_log_price)*1000)                           #In 1970
estimated_price_latest = np.around(estimated_price * SCALE_FACTOR, -3)                             #In 2020

upper_bound = round((np.e**log_upper_bound)*1000)                                   #In 1970
upper_bound_latest = np.around(upper_bound * SCALE_FACTOR, -3)                                     #In 2020

lower_bound = round((np.e**log_lower_bound)*1000)                                   #In 1970
lower_bound_latest = np.around(lower_bound*SCALE_FACTOR, -3)                                       #In 2020

print('Price in 2020 for house esimate: $',estimated_price_latest)
print('Price in 2020 for maximum house price: $',upper_bound_latest)
print('Price in 2020 for minimum house price: $',lower_bound_latest)


print('Price in 1970 for house estimate: $',estimated_price)
print('Price in 1970 for maximum house price: $',upper_bound)
print('Price in 1970 for minimum house price: $',lower_bound)

Price in 2020 for house esimate: $ 827000.0
Price in 2020 for maximum house price: $ 997000.0
Price in 2020 for minimum house price: $ 685000.0
Price in 1970 for house estimate: $ 30047
Price in 1970 for maximum house price: $ 36245
Price in 1970 for minimum house price: $ 24910


In [14]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """
    
    Keyword Arguments:
    rm -- number of rooms in a property
    ptratio -- pupil per teacher in the locality of the property
    chas -- True if property is near Charles river, False otherwise
    large_range -- True for 95% prediction interval, False for 68% prediction interval
    
    """
    
    if rm<1 or ptratio<1:
        print('That is unrealistic, please try again')
        return
    
    estimated_log_price, log_upper_bound, log_lower_bound, interval = get_log_estimate(rm, ptratio, next_to_river=chas,
                                                                                   high_confidence=large_range)
    
    estimated_price = round((np.e**estimated_log_price)*1000)                           #In 1970
    estimated_price_latest = np.around(estimated_price * SCALE_FACTOR, -3)                             #In 2020

    upper_bound = round((np.e**log_upper_bound)*1000)                                   #In 1970
    upper_bound_latest = np.around(upper_bound * SCALE_FACTOR, -3)                                     #In 2020

    lower_bound = round((np.e**log_lower_bound)*1000)                                   #In 1970
    lower_bound_latest = np.around(lower_bound*SCALE_FACTOR, -3)                                       #In 2020

    print('Price in 2020 for house esimate: $',estimated_price_latest)
    print('Price in 2020 for maximum house price: $',upper_bound_latest)
    print('Price in 2020 for minimum house price: $',lower_bound_latest)

In [15]:
get_dollar_estimate(0, 20, chas=True)

That is unrealistic, please try again


In [16]:
import boston_valuation as vals
vals.get_dollar_estimate(4, 13, True)

Price in 2020 for house esimate: $ 629000.0
Price in 2020 for maximum house price: $ 915000.0
Price in 2020 for minimum house price: $ 432000.0
